## 列出从LeNet到ResNet的网络的变化和区别

### 1. LeNet-5模型
##### 总结自https://blog.csdn.net/xjy104165/article/details/78218057;
##### https://www.jianshu.com/p/cd73bc979ba9
LeNet-5模型是Yann LeCun教授于1998年在论文Gradient-based learning applied to document recognition中提出的，它是第一个成功应用于数字识别问题的卷积神经网络。LeNet-5模型一共有7层，下图展示了LeNet-5模型的架构：

![img](https://img-blog.csdn.net/20171012155122043?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvWEpZMTA0MTY1/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

#### 第一层，卷积层
这一层的输入就是原始的图像像素32x32。第一个卷积层过滤器尺寸为5x5，深度为6，不使用全0填充，步长为1。所以这一层的输出：大小为28*28，数量为6个的feature map。
#### 第二层，池化层
这一层的输入为第一层的输出，是一个28x28x6的节点矩阵。本层采用的过滤器大小为2x2，长和宽的步长均为2，所以本层的输出矩阵大小为14x14x6。

(Pooling层的主要作用就是减少数据，降低数据纬度的同时保留最重要的信息。在数据减少后，可以减少神经网络的纬度和计算量，可以防止参数太多过拟合。)
#### LeNet第三层（卷积运算）
本层的输入矩阵大小为14x14x6，使用的过滤器大小为5x5，深度为16.本层不使用全0填充，步长为1。本层的输出矩阵大小为10x10x16。
#### 第四层，池化层
本层的输入矩阵大小10x10x16。本层采用的过滤器大小为2x2，长和宽的步长均为2，所以本层的输出矩阵大小为5x5x16。
#### 第五层，全连接层
本层的输入矩阵大小为5x5x16，在LeNet-5论文中将这一层成为卷积层，但是因为过滤器的大小就是5x5，所以和全连接层没有区别。本层的输出节点个数为120。
#### 第六层，全连接层
本层的输入节点个数为120个，输出节点个数为84个。
#### 第七层，全连接层
本层的输入节点个数为84个，输出节点个数为10个。
#### 激活函数
out为RBF，其余都为tanh

-------------------------------------------------------------------------------------------------

### 2. AlexNet
2012
##### 总结自https://www.cnblogs.com/wangguchangqing/p/10333370.html
AlexNet是在LeNet的基础上加深了网络的结构，学习更丰富更高维的图像特征。AlexNet的特点：
- 更深的网络结构
- 使用层叠的卷积层，即卷积层+卷积层+池化层来提取图像的特征
- 使用Dropout抑制过拟合
- 使用数据增强Data Augmentation抑制过拟合
- 使用Relu替换之前的sigmoid的作为激活函数
- 多GPU训练

#### 新技术：
#### 1. 数据增强
神经网络由于训练的参数多，表能能力强，所以需要比较多的数据量，不然很容易过拟合。当训练数据有限时，可以通过一些变换从已有的训练数据集中生成一些新的数据，以快速地扩充训练数据。对于图像数据集来说，可以对图像进行一些形变操作：

- 翻转
- 随机裁剪
- 平移，颜色光照的变换
- ...

AlexNet中对数据做了以下操作：

- 1.随机裁剪，对256×256的图片进行随机裁剪到227×227，然后进行水平翻转。
- 2.测试的时候，对左上、右上、左下、右下、中间分别做了5次裁剪，然后翻转，共10个裁剪，之后对结果求平均。
- 3.对RGB空间做PCA（主成分分析），然后对主成分做一个（0, 0.1）的高斯扰动，也就是对颜色、光照作变换，结果使错误率又下降了1%。

#### 2. 层叠池化
此前CNN中普遍使用平均池化，AlexNet全部使用最大池化，避免平均池化的模糊化效果。

在LeNet中池化是不重叠的，即池化的窗口的大小和步长是相等的。

在AlexNet中使用的池化（Pooling）却是可重叠的，也就是说，在池化的时候，每次移动的步长小于池化的窗口长度。AlexNet池化的大小为3×3的正方形，每次池化移动步长为2，这样就会出现重叠。重叠池化可以避免过拟合，这个策略贡献了0.3%的Top-5错误率。与非重叠方案s=2，z=2相比，输出的维度是相等的，并且能在一定程度上抑制过拟合。

#### 3. ReLU
#### 4. dropout
#### 5.局部响应归一化（https://www.jianshu.com/p/c014f81242e7)
提出了LRN层，对局部神经元的活动创建竞争机制，使得其中响应比较大的值变得相对更大，并抑制其他反馈较小的神经元，增强了模型的泛化能力。

#### 网络结构
![img](https://img2018.cnblogs.com/blog/439761/201901/439761-20190129114344192-623663293.jpg)

网络包含8个带权重的层；前5层是卷积层，剩下的3层是全连接层。最后一层全连接层的输出是1000维softmax的输入，softmax会产生1000类标签的分布。

- 卷积层C1

    该层的处理流程是： 卷积-->ReLU-->池化-->归一化。
    
    - 卷积，输入是224×224，使用96个11×11×4的卷积核，得到的FeatureMap为55×55×96。
    - ReLU，将卷积层输出的FeatureMap输入到ReLU函数中。
    - 池化，使用3×3步长为2的池化单元（重叠池化，步长小于池化单元的宽度），输出为27×27×96（(55−3)/2+1=27)
    - 局部响应归一化，使用k=2,n=5,α=10−4,β=0.75进行局部归一化，输出的仍然为27×27×96，输出分为两组，每组的大小为27×27×48
- 卷积层C2
    
    该层的处理流程是：卷积-->ReLU-->池化-->归一化
    - 卷积，输入是2组27×27×48。使用2组，每组128个尺寸为5×5×48的卷积核，并作了边缘填充padding=2，卷积的步长为1. 则输出的FeatureMap为2组，每组的大小为27×27 times128. （(27+2∗2−5)/1+1=27）
    - ReLU，将卷积层输出的FeatureMap输入到ReLU函数中
    - 池化运算的尺寸为3×3，步长为2，池化后图像的尺寸为(27−3)/2+1=13，输出为13×13×256
    - 局部响应归一化，使用k=2,n=5,α=10−4,β=0.75进行局部归一化，输出的仍然为13×13×256，输出分为2组，每组的大小为13×13×128
- 卷积层C3
    该层的处理流程是： 卷积-->ReLU
    - 卷积，输入是13×13×256，使用2组共384尺寸为3×3×256的卷积核，做了边缘填充padding=1，卷积的步长为1.则输出的FeatureMap为13×13 times384
    - ReLU，将卷积层输出的FeatureMap输入到ReLU函数中
- 卷积层C4
    该层的处理流程是： 卷积-->ReLU
    该层和C3类似。
    - 卷积，输入是13×13×384，分为两组，每组为13×13×192.使用2组，每组192个尺寸为3×3×192的卷积核，做了边缘填充padding=1，卷积的步长为1.则输出的FeatureMap为13×13 times384，分为两组，每组为13×13×192
    - ReLU，将卷积层输出的FeatureMap输入到ReLU函数中
- 卷积层C5
    该层处理流程为：卷积-->ReLU-->池化
    - 卷积，输入为13×13×384，分为两组，每组为13×13×192。使用2组，每组为128尺寸为3×3×192的卷积核，做了边缘填充padding=1，卷积的步长为1.则输出的FeatureMap为13×13×256
    - ReLU，将卷积层输出的FeatureMap输入到ReLU函数中
    - 池化，池化运算的尺寸为3×3，步长为2，池化后图像的尺寸为 (13−3)/2+1=6,即池化后的输出为6×6×256
- 全连接层FC6
    该层的流程为：（卷积）全连接 -->ReLU -->Dropout
    - 卷积->全连接： 输入为6×6×256,该层有4096个卷积核，每个卷积核的大小为6×6×256。由于卷积核的尺寸刚好与待处理特征图（输入）的尺寸相同，即卷积核中的每个系数只与特征图（输入）尺寸的一个像素值相乘，一一对应，因此，该层被称为全连接层。由于卷积核与特征图的尺寸相同，卷积运算后只有一个值，因此，卷积后的像素层尺寸为4096×1×1，即有4096个神经元。
    - ReLU,这4096个运算结果通过ReLU激活函数生成4096个值
    - Dropout,抑制过拟合，随机的断开某些神经元的连接或者是不激活某些神经元
- 全连接层FC7
    流程为：全连接-->ReLU-->Dropout
    - 全连接，输入为4096的向量
    - ReLU,这4096个运算结果通过ReLU激活函数生成4096个值
    - Dropout,抑制过拟合，随机的断开某些神经元的连接或者是不激活某些神经元
- 输出层
    第七层输出的4096个数据与第八层的1000个神经元进行全连接，经过训练后输出1000个float型的值，这就是预测结果。

------------------------------------------------------------------------------------------

### 3. VGG
##### 总结自https://zhuanlan.zhihu.com/p/41423739
##### 2014, The First Real Deep Neural Networks
VGG是Oxford的Visual Geometry Group的组提出的.主要工作是证明了增加网络的深度能够在一定程度上影响网络最终的性能。VGG有两种结构，分别是VGG16和VGG19，两者并没有本质上的区别，只是网络深度不一样。

VGG16相比AlexNet的一个改进是采用连续的几个3x3的卷积核代替AlexNet中的较大卷积核（11x11，7x7，5x5）。采用堆积的小卷积核是优于采用大的卷积核，因为多层非线性层可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。

网络结构
![img](https://pic4.zhimg.com/80/v2-ea924e733676e0da534f677a97c98653_hd.jpg)

VGG优点
- VGGNet的结构非常简洁，整个网络都使用了同样大小的卷积核尺寸（3x3）和最大池化尺寸（2x2）。
- 几个小滤波器（3x3）卷积层的组合比一个大滤波器（5x5或7x7）卷积层好：
- 验证了通过不断加深网络结构可以提升性能。

VGG缺点
- VGG耗费更多计算资源，并且使用了更多的参数（这里不是3x3卷积的锅），导致更多的内存占用（140M）。其中绝大多数的参数都是来自于第一个全连接层。VGG可是有3个全连接层啊！
PS：有的文章称：发现这些全连接层即使被去除，对于性能也没有什么影响，这样就显著降低了参数数量。

-----------------------------------------------------------------------------------------

### 4. GoogLeNet

总结自https://blog.csdn.net/shuzfan/article/details/50738394

#### Incepetion V1

这是GoogLeNet的最早版本，出现在2014年的《Going deeper with convolutions》。

##### Motivation
深度学习以及神经网络快速发展，人们不再只关注更给力的硬件、更大的数据集、更大的模型，而是更在意新的idea、新的算法以及模型的改进。

一般来说，提升网络性能最直接的办法就是增加网络深度和宽度，这也就意味着巨量的参数。但是，巨量参数容易产生过拟合也会大大增加计算量。

文章认为解决上述两个缺点的根本方法是将全连接甚至一般的卷积都转化为稀疏连接。所以，现在的问题是有没有一种方法，既能保持网络结构的稀疏性，又能利用密集矩阵的高计算性能。大量的文献表明可以将稀疏矩阵聚类为较为密集的子矩阵来提高计算性能，据此论文提出了名为Inception 的结构来实现此目的。

![img](https://img-blog.csdn.net/20160225155351172)

### 网络结构
![img](https://img-blog.csdn.net/20160225155414702)

对上图做如下说明：
1 . 显然GoogLeNet采用了模块化的结构，方便增添和修改；
2 . 网络最后采用了average pooling来代替全连接层，想法来自NIN,事实证明可以将TOP1 accuracy提高0.6%。但是，实际在最后还是加了一个全连接层，主要是为了方便以后大家finetune；
3 . 虽然移除了全连接，但是网络中依然使用了Dropout ;
4 . 为了避免梯度消失，网络额外增加了2个辅助的softmax用于向前传导梯度。文章中说这两个辅助的分类器的loss应该加一个衰减系数，但看caffe中的model也没有加任何衰减。此外，实际测试的时候，这两个额外的softmax会被去掉。

### GoogLeNet Inception V2

《Rethinking the Inception Architecture for Computer Vision》http://arxiv.org/abs/1512.00567

-----------------------------------------------------------------------------------------

### 5. ResNet
总结自https://blog.csdn.net/lanran2/article/details/79057994

2015,ImageNet比赛classification任务上获得第一名

随着网络的加深，出现了训练集准确率下降的现象，我们可以确定这不是由于Overfit过拟合造成的(过拟合的情况训练集应该准确率很高)；所以作者针对这个问题提出了一种全新的网络，叫深度残差网络，它允许网络尽可能的加深，其中引入了全新的结构<font color=black>**Shortcut Connection**</font>
![img](https://img-blog.csdn.net/20180114184946861?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvbGFucmFuMg==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [5]:
pickle_file = '../Lesson-09/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    # tf.truncated_normal(shape, mean, stddev) :shape表示生成张量的维度，mean是均值，stddev是标准差。
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.826666
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.501440
Minibatch accuracy: 75.0%
Validation accuracy: 56.4%
Minibatch loss at step 100: 0.712555
Minibatch accuracy: 75.0%
Validation accuracy: 70.8%
Minibatch loss at step 150: 0.538058
Minibatch accuracy: 87.5%
Validation accuracy: 76.2%
Minibatch loss at step 200: 0.925651
Minibatch accuracy: 68.8%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.930563
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.919396
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 350: 1.221372
Minibatch accuracy: 68.8%
Validation accuracy: 76.4%
Minibatch loss at step 400: 0.678603
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 450: 1.105307
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 500: 0.520046
Minibatch accuracy: 87.5%
Validation accuracy: 80.4%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---